In [43]:
import advent
import math
advent.scrape(2024, 17)
data = advent.get_lines_doublenewline(17)
registers = {'A': 45483412, 'B': 0, 'C': 0}
program = [int(i) for i in data[1][0][9:].split(",")]

In [44]:
def combo(registers: dict[str, int], operand: int) -> int:
    if operand < 4: return operand
    if operand < 7: return registers[chr(operand + 61)]
    raise ValueError(operand)

def step(registers: dict[str, int], instruction_pointer: int):
    opcode, operand = program[instruction_pointer], program[instruction_pointer + 1]
    out = []
    if opcode == 0:
        registers['A'] = registers['A'] >> combo(registers, operand)
    elif opcode == 1:
        registers['B'] = registers['B'] ^ operand
    elif opcode == 2:
        registers['B'] = combo(registers, operand) % 8
    elif opcode == 3:
        if registers['A'] != 0: instruction_pointer = (operand - 2)
    elif opcode == 4:
        registers['B'] = registers['B'] ^ registers['C']
    elif opcode == 5:
        out = [combo(registers, operand) % 8]
    elif opcode == 6:
        registers['B'] = registers['A'] >> combo(registers, operand)
    elif opcode == 7:
        registers['C'] = registers['A'] >> combo(registers, operand)
    instruction_pointer += 2
    return registers, instruction_pointer, out

In [45]:
instruction_pointer, result = 0, []
registers = {'A': 45483412, 'B': 0, 'C': 0}

while instruction_pointer < (len(program) - 1):
    registers, instruction_pointer, out = step(registers, instruction_pointer)
    result = result + out
    if instruction_pointer == 0:
        print(registers, result)
print(",".join([str(c) for c in result]))

{'A': 5685426, 'B': 355337, 'C': 355339} [1]
{'A': 710678, 'B': 2842717, 'C': 2842713} [1, 5]
{'A': 88834, 'B': 22208, 'C': 22208} [1, 5, 0]
{'A': 11104, 'B': 44421, 'C': 44417} [1, 5, 0, 5]
{'A': 1388, 'B': 1386, 'C': 1388} [1, 5, 0, 5, 2]
{'A': 173, 'B': 8, 'C': 10} [1, 5, 0, 5, 2, 0]
{'A': 21, 'B': 1, 'C': 2} [1, 5, 0, 5, 2, 0, 1]
{'A': 2, 'B': 3, 'C': 0} [1, 5, 0, 5, 2, 0, 1, 3]
1,5,0,5,2,0,1,3,5


In [108]:
from tqdm import tqdm

# Brute force, fairly hopeless...

program = [0,3,5,4,3,0] # This works very fast
program = [2,4,1,3,7,5,0,3,4,1,1,5,5,5,3,0]

def run(i):
    registers = {'A': i, 'B': 0, 'C': 0}
    instruction_pointer = 0
    output_ix = 0
    while instruction_pointer < (len(program) - 1):
        registers, instruction_pointer, out = step(registers, instruction_pointer)
        if out:
            if out[0] != program[output_ix]: return False
            output_ix += 1
    return output_ix == len(program)

#for i in tqdm(range(0, 8**16)):
#    if run(i):
#        break
#print(i)


In [109]:
# Part 2, honestly I didn't even want to try brute forcing it, seems like a lost cause. (I wrote the cell above later just to prove it wouldnt work)

# Combo: 4=a,5=b,6=c
# The program is: 2,4 - 1,3 - 7,5 - 0,3 - 4,1 - 1,5 - 5,5 - 3,0
# B = A % 8
# B = B xor b'011'
# C = A >> B
# A = A >> 3
# B = B xor C
# B = B xor b'101'
# output B % 8
# if A != 0, jump to beginning

# We can 'solve' it a bit: the output is B xor C xor 101, where B = A%8 xor 011, and C = A >> B
# so [ A%8 xor (A >> (A%8 xor 101)) xor 110 ]    (110 = 101 xor 011)
# Then after each iteration, A = A >> 3, and B and C are overwritten again

# To keep it simple, let's say we want the first output to be 0, 1, 0
# and e.g. A%8 = 0. Then we enforce that A >> 5 xor 110 must be 010, so A>>5 must be 100
# But let's say A%8 = 7, then A >> 2 xor 110 must be 101, so A>>2 must be 011
# etcetera. We get these 'branches' that simultaneously have some free choices while also forcing some bits

In [129]:
# But we get a lot more constrained towards the end: the last output is 0, and we know there cannot be any more bits above that
# (or else it will output more stuff at the end)
# So the restriction must be some A >> n = 0, or perhaps A >> 0 = xxx, or A >> 1 = 0xx, or A >> 2 = 00x

# A%8 = xxx --> A >> (101 xor xxx) = (110 xor xxx)
from typing import Iterator

def determine_next_number(A = 0, next = 0) -> Iterator[int]:
    for i in range(8):
        As = 8*A + i
        B = (As % 8) ^ 3
        C = (As >> B)
        o = (B ^ C ^ 5) % 8
        # I'm not sure why I was so focused on this C == i^6 check, cost me a lot of time and removing it just worked
        if o == next: # and C == i^6:
            yield i
        #print(f"A%8 = {format(i, '03b',)} --> A >> {i^5} ({format(i >> (i^5), '03b')}) = {format(i^6, '03b')}. output = {format(o, '03b')} ({format(next, '03b')})")

def determine_all_numbers(A: int, nexts: list[int]) -> Iterator[list[int]]:
    # Recursively try every possibile number and continue down the list until we reach empty list, which is always possible
    # We build A 'top-down' i.e. starting with most significant digit, and working our way down to less significant digits
    # input: A is all the most significant digits decided so far, nexsts is the list of desired outputs
    # Because we go in 'reverse' order, the last element of nexts is the one we actually want to output 'first'
    if len(nexts) == 0:
        yield []
        return
    numbers = list(determine_next_number(A, nexts[-1]))
    for num in numbers:
        An = 8*A + num
        for r in determine_all_numbers(An, nexts[:-1]):
            yield [num] + r


program = [2,4,1,3,7,5,0,3,4,1,1,5,5,5,3,0]
for i in determine_all_numbers(0, program):
    print(int(''.join([str(j) for j in i]), 8))

236581108670061
236581108670143
